<a href="https://colab.research.google.com/github/UniqueUo16/Ethical-/blob/main/Viewr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from flask import Flask, render_template_string, request, redirect, url_for
import json, random, os
import threading
from pyngrok import ngrok

app = Flask(__name__)
DATA_FILE = "players.json"

# Initialize JSON storage
if not os.path.exists(DATA_FILE):
    with open(DATA_FILE, "w") as f:
        json.dump({}, f)

def load_players():
    with open(DATA_FILE, "r") as f:
        return json.load(f)

def save_players(players):
    with open(DATA_FILE, "w") as f:
        json.dump(players, f, indent=4)

START_BALANCE = 5000

# ----------------- Templates -----------------
base_html = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>{{ title if title else "Bet App" }}</title>
<style>
body { background: #0b0b0b; color: #fff; font-family: 'Arial', sans-serif; text-align:center; }
nav a { color:#0ff; margin:0 10px; text-decoration:none; font-weight:bold; }
button { background: linear-gradient(90deg, #0ff, #0f0); color:#111; padding:10px 20px; border:none; border-radius:10px; cursor:pointer; box-shadow:0 0 20px #0ff,0 0 40px #0f0; }
button:hover { transform: scale(1.05); }
.win { color:#0f0; font-weight:bold; text-shadow:0 0 10px #0f0; }
.loss { color:#f00; font-weight:bold; text-shadow:0 0 10px #f00; }
.jackpot { color: gold; font-weight:bold; text-shadow:0 0 20px gold,0 0 40px orange; }
#reel-display { font-size:2em; margin:20px 0; }
ul { list-style:none; padding:0; }
ul li.win { color:#0f0; }
ul li.loss { color:#f00; }
</style>
</head>
<body>
<nav>
<a href="{{ url_for('dashboard') }}">Dashboard</a> |
<a href="{{ url_for('leaderboard') }}">Leaderboard</a>
</nav>
<hr>
{% block content %}{% endblock %}
<script>
// Slot JS + sounds + confetti
document.addEventListener("DOMContentLoaded",()=>{
    const spinBtn = document.querySelector("button[type='submit']");
    const resultMsg = document.querySelector("p.win, p.loss, p.jackpot");
    spinBtn?.addEventListener("click",(e)=>{
        const spinSound = new Audio("/static/spin.mp3");
        spinSound.play();
        const reels = ["🍒","🍋","🍊","7️⃣","⭐"];
        const display = document.getElementById("reel-display");
        let count=0;
        const interval=setInterval(()=>{
            display.innerText = `${reels[Math.floor(Math.random()*reels.length)]} ${reels[Math.floor(Math.random()*reels.length)]} ${reels[Math.floor(Math.random()*reels.length)]}`;
            count++;
            if(count>10) clearInterval(interval);
        },100);

        setTimeout(()=>{
            if(resultMsg){
                if(resultMsg.classList.contains("win")){
                    new Audio("/static/win.mp3").play();
                    confetti({particleCount:100,spread:70});
                } else if(resultMsg.classList.contains("jackpot")){
                    new Audio("/static/jackpot.mp3").play();
                    confetti({particleCount:200,spread:100});
                }
            }
        },1200);

        function confetti(opt={particleCount:100,spread:70}){
            const script=document.createElement("script");
            script.src="https://cdn.jsdelivr.net/npm/canvas-confetti@1.6.0/dist/confetti.browser.min.js";
            script.onload=()=>confettiLib(opt);
            document.body.appendChild(script);
            function confettiLib(opt){confetti(opt);}
        }
    });
});
</script>
</body>
</html>
"""

dashboard_html = """
{% extends "base.html" %}
{% block content %}
<h1>🏆 Dashboard</h1>
<ul>
{% for username, player in players.items() %}
<li>
<a href="{{ url_for('profile', username=username) }}">{{ player.name }}</a>
- Balance: ${{ player.balance }} - Wins: {{ player.wins }}
<a href="{{ url_for('play', username=username) }}">Play</a>
</li>
{% else %}
<li>No players yet!</li>
{% endfor %}
</ul>
{% endblock %}
"""

profile_html = """
{% extends "base.html" %}
{% block content %}
<h1>👤 Profile: {{ player.name }}</h1>
<p>Balance: ${{ player.balance }} | Wins: {{ player.wins }} | Losses: {{ player.losses }} | Streak: {{ player.streak }}</p>
<h3>Last Spins:</h3>
<ul>
{% for h in player.history %}
<li class="{{ h }}">{{ h|capitalize }}</li>
{% endfor %}
</ul>
{% endblock %}
"""

play_html = """
{% extends "base.html" %}
{% block content %}
<h1>🎰 Slot Machine - {{ player.name }}</h1>
<p>Balance: ${{ player.balance }} | Wins: {{ player.wins }} | Streak: {{ player.streak }}</p>
<form method="POST">
<label>Stake:</label>
<input type="number" name="stake" value="100" min="10" max="{{ player.balance }}">
<button type="submit">Spin</button>
</form>
<div id="reel-display"></div>
{% if result %}
<p class="{{ result.type }}">{{ result.msg }}</p>
{% endif %}
<h3>Last Spins:</h3>
<ul>
{% for h in player.history %}
<li class="{{ h }}">{{ h|capitalize }}</li>
{% endfor %}
</ul>
{% endblock %}
"""

leaderboard_html = """
{% extends "base.html" %}
{% block content %}
<h1>🏅 Leaderboard</h1>
<ol>
{% for player in players %}
<li>{{ player.name }} - ${{ player.balance }} - Wins: {{ player.wins }}</li>
{% endfor %}
</ol>
{% endblock %}
"""

# ----------------- Routes -----------------
@app.route("/")
def dashboard():
    players = load_players()
    return render_template_string(dashboard_html, players=players)

@app.route("/profile/<username>")
def profile(username):
    players = load_players()
    player = players.get(username)
    if not player:
        players[username] = {
            "name": username,
            "balance": START_BALANCE,
            "wins":0, "losses":0, "streak":0, "history":[]
        }
        save_players(players)
        player = players[username]
    return render_template_string(profile_html, player=player)

@app.route("/play/<username>", methods=["GET","POST"])
def play(username):
    players = load_players()
    player = players.get(username)
    if not player: return redirect(url_for("dashboard"))
    result = None
    if request.method=="POST":
        stake = int(request.form.get("stake",100))
        if stake > player["balance"]:
            result = {"msg":"Insufficient balance!","type":"loss"}
        else:
            player["balance"] -= stake
            secret=random.randint(1,5)
            guess=random.randint(1,5)
            if guess==secret:
                win_amount = stake*5
                player["balance"] += win_amount
                player["wins"] += 1
                player["streak"] += 1
                result = {"msg":f"You won ${win_amount}!","type":"win"}
            else:
                player["losses"] += 1
                player["streak"] = 0
                result = {"msg":f"You lost ${stake}. Number was {secret}","type":"loss"}
            player["history"].insert(0,result["type"])
            if len(player["history"])>20: player["history"]=player["history"][:20]
            save_players(players)
    return render_template_string(play_html, player=player, result=result)

@app.route("/leaderboard")
def leaderboard():
    players = load_players()
    sorted_players = sorted(players.values(), key=lambda x:x["balance"], reverse=True)
    return render_template_string(leaderboard_html, players=sorted_players[:10])

# ----------------- Run App -----------------
if __name__=="__main__":
    def run():
        app.run(port=5000)

    threading.Thread(target=run).start()

    # Connect ngrok
    public_url = ngrok.connect(5000)
    print("Your public URL:", public_url)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Your public URL: NgrokTunnel: "https://7f17c82f8869.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
# Replace "YOUR_AUTHTOKEN" with your actual ngrok authtoken
NGROK_AUTHTOKEN = "yr_auth_token"
!ngrok authtoken {NGROK_AUTHTOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%pip install pyngrok

In [1]:
!kill $(lsof -t -i:5000)
!kill $(lsof -t -i:4040)

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
